Notebook for analyzing T-38 and weather balloon data simultaneously


Import statements:

In [498]:
import pandas as pd
import numpy as np
import scipy
import matplotlib
import re
from JMOSS import utilities
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator as ML
from matplotlib.ticker import ScalarFormatter as SF

Import test point .csv files

In [499]:

class55 = pd.read_csv('../SampleData/CLASS55.csv')
class65 = pd.read_csv('../SampleData/CLASS65.csv')
class95 = pd.read_csv('../SampleData/CLASS95.csv')

testPoints = [class55,class65,class95]


Import weather balloon

In [500]:
wb_filepath = '../SampleData/WeatherBalloon.txt'

pattern = re.compile(r"\d+\s+\d+\s+\d+.\d+\s+.\d+\s+-*\d+.\d+\s+-*\d+.\d+\s+\d+.\d+\s+\d+\s+\d+.\d+\s+\d+.\d+\s+\d+\s+\d+\s+\d+.\d+\s+\d+")

labels = ['Hg','w_dir','w_mag','SHR','Ta','DPT','Pa','RH','AB_HUM','rho','I_R','V_S','VPS','PW']

f = open(wb_filepath,'r')

lines = f.readlines()

wbdata = []

for line in lines:
    out = re.findall(pattern,line)
    if len(out) > 0:
        data = [float(a) for a in out[0].split()]
        wbdata.append(data)

wbdata = pd.DataFrame(wbdata,columns=labels)

Clean weather balloon dataframe and functions

In [501]:
import math
import scipy.stats as stats

# make an empty DataFrame
balloon_DB = pd.DataFrame()

# Load geometric height in feet
balloon_DB['height_ft'] = wbdata['Hg']

# Change wind magnitude and direction to N and E components in the nav frame
balloon_DB['wind_mag_kts'] = wbdata['w_mag']
balloon_DB['wind_dir_rad'] = wbdata.apply(lambda x:
                                   math.radians(x['w_dir']),axis=1)
balloon_DB['wind_mag_NAV_N_kts'] = wbdata.apply(lambda x:
                           x['w_mag']*(math.cos(math.radians(x['w_dir']))),axis=1)
balloon_DB['wind_mag_NAV_E_kts'] = wbdata.apply(lambda x:
                           x['w_mag']*(math.sin(math.radians(x['w_dir']))),axis=1)
# Check that the transformation was done correctly
balloon_DB['wind_mag_check'] = balloon_DB.apply(lambda x:
                                            x['wind_mag_kts']**2 - x['wind_mag_NAV_N_kts']**2 - x['wind_mag_NAV_E_kts']**2,axis=1)

# Load Pa and Ta in psi and K, respectively
millibar_to_psi = 0.0145037738
celsius_to_kelvin = 274.15
balloon_DB['ambient_pres_psi'] = wbdata.apply(lambda x:
                                x['Pa']*millibar_to_psi,axis=1)
balloon_DB['ambient_temp_K'] = wbdata.apply(lambda x:
                                x['Ta'] + celsius_to_kelvin,axis=1)



Load balloon data into test point time series
Calculate additional needed parameters

In [ ]:
fig, dpaxs = plt.subplots(figsize=(16, 6))
fig2, dTaxs = plt.subplots(figsize=(16, 6))

ival = 0

for testPoint in testPoints:

    # ambient pressure interpolated from balloon_DB
    testPoint['ambient_pres_psi'] = np.interp(testPoint['height_ft'],
                                              balloon_DB['height_ft'],
                                              balloon_DB['ambient_pres_psi'])
    # ambient temperature interpolated from balloon_DB
    testPoint['ambient_temp_K'] = np.interp(testPoint['height_ft'],
                                              balloon_DB['height_ft'],
                                              balloon_DB['ambient_temp_K'])

    # wind component N interpolated from balloon_DB
    testPoint['wind_mag_NAV_N_kts'] = np.interp(testPoint['height_ft'],
                                              balloon_DB['height_ft'],
                                              balloon_DB['wind_mag_NAV_N_kts'])

    # wind component E interpolated from balloon_DB
    testPoint['wind_mag_NAV_E_kts'] = np.interp(testPoint['height_ft'],
                                              balloon_DB['height_ft'],
                                              balloon_DB['wind_mag_NAV_E_kts'])

    # qcic/Ps from recorded data
    testPoint['qcicPs'] = (testPoint['total_pres_psi'] - testPoint['static_pres_psi'])/testPoint['static_pres_psi']

    # qc/Pa from balloon and recorded data
    testPoint['qcPaBalloon'] = (testPoint['total_pres_psi'] - testPoint['ambient_pres_psi'])/testPoint['ambient_pres_psi']

    # Mach_IC from recorded data (qcic/Ps)
    testPoint['MachIC'] = utilities.mach_from_qc_pa(testPoint['qcicPs'].values)

    # Delta P_p/Ps estimates from survey method
    testPoint['DeltaPpPsBalloon'] = -(testPoint['qcicPs'] - testPoint['qcPaBalloon'])/(testPoint['qcPaBalloon']+1)

    # Static temp from total and Mach (indicated)
    testPoint['static_temp_K'] = testPoint['total_temp_K']/(1+0.2*testPoint['MachIC']**2)

    # Survey "DeltaT/T
    testPoint['M2over5'] = (testPoint['MachIC']**2)/5
    testPoint['TicTa1'] = testPoint['total_temp_K']/testPoint['ambient_temp_K'] - 1

    # plot DeltaP_p/P_s results
    testPoint.plot.scatter(x="MachIC",y="DeltaPpPsBalloon",
                           ax=dpaxs,
                           color=['C'+str(ival)],
                           marker='.')
    # axis configuration for above
    dpaxs.set_xlabel('$M_{ic}$')
    dpaxs.set_ylabel('$\Delta P_p/P_s$')
    dpaxs.xaxis.set_minor_locator(ML(0.05))
    dpaxs.xaxis.set_minor_formatter(SF())
    dpaxs.tick_params(axis='x',which='minor')
    dpaxs.grid('on','both')
    dpaxs.set_title('Survey Method Results')

    # plot Kt results
    testPoint.plot.scatter(x="M2over5",y="TicTa1",
                           ax=dTaxs,
                           color=['C'+str(ival)],
                           marker='.')

    # axis configuration for above
    dTaxs.set_xlabel('$0.2*M_{ic}^2$')
    dTaxs.set_ylabel('$T_{ic}/T_a - 1$')
    dTaxs.xaxis.set_minor_locator(ML(0.05))
    dTaxs.xaxis.set_minor_formatter(SF())
    dTaxs.tick_params(axis='x',which='minor')
    dTaxs.grid('on','both')
    dTaxs.set_title('Survey Method Results')

    # increment color wheel
    ival += 1


Load auxiliary helper functions


Build Kalman filter equations

In [503]:
# use example below

from pykalman import AdditiveUnscentedKalmanFilter

N_states = 5
# STATES
# Wind N
# Wind E
# Wind D
# DPp/Ps
# Ta
# ADD CONTROL PARAMETERS AND CONVERT TO LAMBDA FUNCTION AT EACH TIME STEP
def additive_transition_function(state):
    return #TBD

# MEASUREMENTS
# GPS Velocity Vector = Wind Velocity Vector + Pitot-Static Velocity Vector
# Balloon Wind N = Wind N
# Balloon Wind E = Wind E
# Balloon Pa = Ps - DPp/Ps*Ps
# Balloon Ta = Ta
def additive_observation_function(state):
    wind = state[0:2]
    mach_ic = mach_from_qc_pa
    tas_w = mach_pc * sqrt(oat / 288.1500) * 661.478827231622
    tas_n = wind_to_nav.apply(column_stack([tas_w, 0 * tas_w, 0 * tas_w]))
    gs_n_est = tas_n + wind

    return #TBD

transition_covariance = np.eye(N_states)
random_state = np.random.RandomState(0)
observation_covariance = np.eye(2) + random_state.randn(2, 2) * 0.1
initial_state_mean = [0, 0]
initial_state_covariance = [[1, 0.1], [ 0.1, 1]]

# sample from model
ukf = UnscentedKalmanFilter(
    transition_function, observation_function,
    transition_covariance, observation_covariance,
    initial_state_mean, initial_state_covariance,
    random_state=random_state
)
akf = AdditiveUnscentedKalmanFilter(
    additive_transition_function, additive_observation_function,
    transition_covariance, observation_covariance,
    initial_state_mean, initial_state_covariance
)
states, observations = ukf.sample(50, initial_state_mean)

# estimate state with filtering
ukf_state_estimates = ukf.filter(observations)[0]
akf_state_estimates = akf.filter(observations)[0]

# draw estimates
pl.figure()
lines_true = pl.plot(states, color='b')
lines_ukf = pl.plot(ukf_state_estimates, color='r', ls='-')
lines_akf = pl.plot(akf_state_estimates, color='g', ls='-.')
pl.legend((lines_true[0], lines_ukf[0], lines_akf[0]),
          ('true', 'UKF', 'AddUKF'),
          loc='upper left'
)
pl.show()


In [ ]:
# use this example and run algo online

import numpy as np
import pylab as pl

from pykalman.datasets import load_robot
from pykalman import KalmanFilter

# Initialize the Kalman Filter
data = load_robot()
kf = KalmanFilter(
    data.transition_matrix,
    data.observation_matrix,
    data.initial_transition_covariance,
    data.initial_observation_covariance,
    data.transition_offsets,
    data.observation_offset,
    data.initial_state_mean,
    data.initial_state_covariance,
    random_state=0
)

# Estimate mean and covariance of hidden state distribution iteratively.  This
# is equivalent to
#
#   >>> (filter_state_means, filtered_state_covariance) = kf.filter(data)
n_timesteps = data.observations.shape[0]
n_dim_state = data.transition_matrix.shape[0]
filtered_state_means = np.zeros((n_timesteps, n_dim_state))
filtered_state_covariances = np.zeros((n_timesteps, n_dim_state, n_dim_state))
for t in range(n_timesteps - 1):
    if t == 0:
        filtered_state_means[t] = data.initial_state_mean
        filtered_state_covariances[t] = data.initial_state_covariance
    filtered_state_means[t + 1], filtered_state_covariances[t + 1] = (
        kf.filter_update(
            filtered_state_means[t],
            filtered_state_covariances[t],
            data.observations[t + 1],
            transition_offset=data.transition_offsets[t],
        )
    )

# draw estimates
pl.figure()
lines_true = pl.plot(data.states, color='b')
lines_filt = pl.plot(filtered_state_means, color='r')
pl.legend((lines_true[0], lines_filt[0]), ('true', 'filtered'))
pl.show()